In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -q datasets
from datasets import load_dataset

qpn_dataset = load_dataset('ShoAnn/indo_legalqa_triplets')
qpn_dataset

DatasetDict({
    train: Dataset({
        features: ['q', 'pos', 'neg'],
        num_rows: 7037
    })
    validation: Dataset({
        features: ['q', 'pos', 'neg'],
        num_rows: 610
    })
    test: Dataset({
        features: ['q', 'pos', 'neg'],
        num_rows: 610
    })
})

In [17]:
from sentence_transformers import SentenceTransformer, SentenceTransformerTrainer, losses
from sentence_transformers.training_args import SentenceTransformerTrainingArguments
import wandb

# model = SentenceTransformer("indobenchmark/indobert-base-p1")
model = SentenceTransformer("indobenchmark/indobert-base-p1")

loss = losses.TripletLoss(model=model)

training_args = SentenceTransformerTrainingArguments(
    output_dir="/content/drive/MyDrive/model_outputs/retriever_checkpoints",
    num_train_epochs=10,
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    per_device_eval_batch_size=8,
    logging_steps=100,
    save_steps=200,
    save_total_limit=2,
    eval_strategy="steps",
    eval_steps=200,
    # Load the best model at the end of training
    load_best_model_at_end=True
)

trainer = SentenceTransformerTrainer(
    model=model,
    args=training_args,
    train_dataset=qpn_dataset["train"],
    eval_dataset=qpn_dataset["validation"],
    loss=loss,
)

ImportError: cannot import name 'Transformers' from 'sentence_transformers.models' (/usr/local/lib/python3.11/dist-packages/sentence_transformers/models/__init__.py)

In [18]:
trainer.train()

Step,Training Loss,Validation Loss
200,0.335200,0.014311


Step,Training Loss,Validation Loss
200,0.335200,0.014311
400,0.051800,0.000500
600,0.068400,0.000000
800,0.055200,0.015275
1000,0.011800,0.000000
1200,0.003100,0.004626
1400,0.008200,0.005834
1600,0.000000,0.004959
1800,0.000000,0.004959
2000,0.005100,0.002249


TrainOutput(global_step=2200, training_loss=0.05062684232538397, metrics={'train_runtime': 12526.1416, 'train_samples_per_second': 5.618, 'train_steps_per_second': 0.176, 'total_flos': 0.0, 'train_loss': 0.05062684232538397, 'epoch': 10.0})

In [19]:
# save to hf hub
from google.colab import userdata
trainer.push_to_hub(token=userdata.get('HF_TOKEN'))

events.out.tfevents.1746615491.eea804e972b9.368.1:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

events.out.tfevents.1746615367.eea804e972b9.368.0:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

events.out.tfevents.1746658843.3b9e6a73703a.2734.0:   0%|          | 0.00/16.8k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.62k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ShoAnn/retriever_checkpoints/commit/fe5bc57ec0aa034bdf633baa83d3526fa1ab2b44', commit_message='End of training', commit_description='', oid='fe5bc57ec0aa034bdf633baa83d3526fa1ab2b44', pr_url=None, repo_url=RepoUrl('https://huggingface.co/ShoAnn/retriever_checkpoints', endpoint='https://huggingface.co', repo_type='model', repo_id='ShoAnn/retriever_checkpoints'), pr_revision=None, pr_num=None)